### Preparation

We generated an AUTH TOKEN and saved it in `auth_token.txt` file which will be used for the rest of this exercise.

In [1]:
def get_auth_token():
    with open('auth_token.txt') as f:
        return f.read().strip()

auth_token = get_auth_token()

In [13]:
# imports
import requests
import json
import time

### Challenge 1: Fork Languages

Assuming the main lab repo is `ironhack-datalabs/bcn-feb-2019`:

**Obtain the full list of forks created from the main lab repo via Github API.**

In [6]:
url = 'https://api.github.com/repos/ironhack-datalabs/bcn-feb-2019/forks'
headers = {
    'Accept': 'application/vnd.github+json',
    'Authorization': f'Bearer {auth_token}',
    'X-GitHub-Api-Version': '2022-11-28'
}
content = requests.get(url, headers=headers).content
forks_raw = json.loads(content)

In [7]:
len(forks_raw)

7

**Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.**

In [8]:
fork_langs = []
for fork in forks_raw:
    lang = fork['language']
    if lang not in fork_langs:
        fork_langs.append(lang)
fork_langs

[None, 'Jupyter Notebook']

### Challenge 2: Count Commits

Assuming the main lab repo is `lobehub/lobe-chat`, you will count how many commits were made in the past 3 weeks:

**NOTE: I changed the repo since the one used in the assignment has no recent commits**

**Obtain all the commits made in the past 3 weeks via API, which is a JSON array that contains multiple commit objects.**

In [15]:
since_date = '2023-11-23T00:00:01Z'
current_page = 1
commits = []

while True:
    url = f'https://api.github.com/repos/lobehub/lobe-chat/commits?since={since_date}&page={current_page}&per_page=100'
    content = requests.get(url, headers=headers).content
    result_list = json.loads(content)
    if len(result_list) == 0:
        break
    commits = commits + result_list
    current_page += 1
    time.sleep(2)
    
commits

[{'sha': 'c295aaf66f26af8237994cc24cdfd92d53980d73',
  'node_id': 'C_kwDOJlox89oAKGMyOTVhYWY2NmYyNmFmODIzNzk5NGNjMjRjZGZkOTJkNTM5ODBkNzM',
  'commit': {'author': {'name': 'Ikko Eltociear Ashimine',
    'email': 'eltociear@gmail.com',
    'date': '2023-12-14T15:30:28Z'},
   'committer': {'name': 'GitHub',
    'email': 'noreply@github.com',
    'date': '2023-12-14T15:30:28Z'},
   'message': '📝 docs: update .env.example (#662)\n\nseperate -> separate',
   'tree': {'sha': '7e17d8cad156252b1d23668f3bf003f10cf2a072',
    'url': 'https://api.github.com/repos/lobehub/lobe-chat/git/trees/7e17d8cad156252b1d23668f3bf003f10cf2a072'},
   'url': 'https://api.github.com/repos/lobehub/lobe-chat/git/commits/c295aaf66f26af8237994cc24cdfd92d53980d73',
   'comment_count': 0,
   'verification': {'verified': True,
    'reason': 'valid',
    'signature': '-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQBQJlex+UCRBK7hj4Ov3rIwAAi08IAFDmHAcNgotv2t7yBO/KOP5U\nN2J6biiiYePP2KZceD7jvVzbkzA/sVpe8DZq/mC2iKPJsR7q9XVASmPi

In [16]:
len(commits)

135